In [9]:
import pandas as pd

# Load dataset
data_path = '../dataset/dataset.xlsx'  # Adjust relative path according to notebook location
df = pd.read_excel(data_path)

# Quick look
print(df.head())
print(df.info())
print(df.describe())
print(df.isnull().sum())


       Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time  Arrival_Time Duration Total_Stops Additional_Info  Price  
0    22:20  01:10 22 Mar   2h 50m    non-stop         No info   3897  
1    05:50         13:15   7h 25m     2 stops         No info   7662  
2    09:25  04:25 10 Jun      19h     2 stops         No info  13882  
3    18:05         23:30   5h 25m      1 stop         No info   6218  
4    16:50         21:35   4h 45m      1 stop         No info  13302  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 

In [10]:
# Drop rows with missing values in 'Route' and 'Total_Stops'
df = df.dropna(subset=['Route', 'Total_Stops'])

# Map 'Total_Stops' to numerical
stops_mapping = {
    'non-stop': 0,
    '1 stop': 1,
    '2 stops': 2,
    '3 stops': 3,
    '4 stops': 4
}
df['Total_Stops'] = df['Total_Stops'].map(stops_mapping)

# Verify changes
print(df['Total_Stops'].value_counts())
print(df.isnull().sum())


Total_Stops
1    5625
0    3491
2    1520
3      45
4       1
Name: count, dtype: int64
Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64


In [11]:
# Convert Date_of_Journey to datetime
df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y')

# Extract day and month
df['journey_day'] = df['Date_of_Journey'].dt.day
df['journey_month'] = df['Date_of_Journey'].dt.month

# Dep_Time - extract hour and minute
df['Dep_Time'] = pd.to_datetime(df['Dep_Time'], format='%H:%M')
df['dep_hour'] = df['Dep_Time'].dt.hour
df['dep_minute'] = df['Dep_Time'].dt.minute

# Arrival_Time - extract hour and minute
# Note: Some Arrival_Time might include date info, handle accordingly
def extract_arrival_time(t):
    try:
        return pd.to_datetime(t, format='%H:%M')
    except:
        return pd.to_datetime(t, format='%H:%M %d %b')

df['Arrival_Time'] = df['Arrival_Time'].apply(extract_arrival_time)
df['arrival_hour'] = df['Arrival_Time'].dt.hour
df['arrival_minute'] = df['Arrival_Time'].dt.minute

# Duration - convert to total minutes
def duration_to_minutes(duration):
    time_parts = duration.split()
    total_mins = 0
    for part in time_parts:
        if 'h' in part:
            total_mins += int(part.replace('h', '')) * 60
        if 'm' in part:
            total_mins += int(part.replace('m', ''))
    return total_mins

df['duration_mins'] = df['Duration'].apply(duration_to_minutes)

# Drop original columns if you want
df = df.drop(['Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration', 'Route'], axis=1)

# Check new columns
df.head()


,Airline,Source,Destination,Total_Stops,Additional_Info,Price,journey_day,journey_month,dep_hour,dep_minute,arrival_hour,arrival_minute,duration_mins
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,22,20,1,10,170
1,Air India,Kolkata,Banglore,2,No info,7662,1,5,5,50,13,15,445
2,Jet Airways,Delhi,Cochin,2,No info,13882,9,6,9,25,4,25,1140
3,IndiGo,Kolkata,Banglore,1,No info,6218,12,5,18,5,23,30,325
4,IndiGo,Banglore,New Delhi,1,No info,13302,1,3,16,50,21,35,285


In [12]:
categorical_columns = ['Airline', 'Source', 'Destination', 'Additional_Info']

for col in categorical_columns:
    print(f"Unique values in '{col}' ({len(df[col].unique())} categories):")
    print(df[col].value_counts())
    print("\n" + "-"*50 + "\n")


Unique values in 'Airline' (12 categories):
Airline
Jet Airways                          3849
IndiGo                               2053
Air India                            1751
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64

--------------------------------------------------

Unique values in 'Source' (5 categories):
Source
Delhi       4536
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: count, dtype: int64

--------------------------------------------------

Unique values in 'Destination' (6 categories):
Destination
Cochin       4536
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kol

In [13]:
# Drop Additional_Info
df = df.drop('Additional_Info', axis=1)

# One-hot encode categorical variables
cat_cols = ['Airline', 'Source', 'Destination']
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Check the new dataframe shape and columns
print("New shape after encoding:", df_encoded.shape)
df_encoded.head()


New shape after encoding: (10682, 29)


,Total_Stops,Price,journey_day,journey_month,dep_hour,dep_minute,arrival_hour,arrival_minute,duration_mins,Airline_Air India,...,Airline_Vistara Premium economy,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0,3897,24,3,22,20,1,10,170,False,...,False,False,False,False,False,False,False,False,False,True
1,2,7662,1,5,5,50,13,15,445,True,...,False,False,False,True,False,False,False,False,False,False
2,2,13882,9,6,9,25,4,25,1140,False,...,False,False,True,False,False,True,False,False,False,False
3,1,6218,12,5,18,5,23,30,325,False,...,False,False,False,True,False,False,False,False,False,False
4,1,13302,1,3,16,50,21,35,285,False,...,False,False,False,False,False,False,False,False,False,True


In [14]:
# Convert bool columns to int
df_encoded = df_encoded.astype({col: 'int' for col in df_encoded.select_dtypes('bool').columns})

# Display all columns (in Jupyter, to see wide DataFrame)
from IPython.display import display
pd.set_option('display.max_columns', None)  # Show all columns
display(df_encoded.head())


,Total_Stops,Price,journey_day,journey_month,dep_hour,dep_minute,arrival_hour,arrival_minute,duration_mins,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0,3897,24,3,22,20,1,10,170,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,7662,1,5,5,50,13,15,445,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2,13882,9,6,9,25,4,25,1140,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
3,1,6218,12,5,18,5,23,30,325,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1,13302,1,3,16,50,21,35,285,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Separate features and target
X = df_encoded.drop('Price', axis=1)
y = df_encoded['Price']

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
rf_model = RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1)

# Train the model
rf_model.fit(X_train, y_train)

# Predictions
y_pred = rf_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


MAE: 1157.62
RMSE: 1962.26


In [16]:
pip install xgboost lightgbm catboost


Note: you may need to restart the kernel to use updated packages.


In [17]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

models = {
    "XGBoost": XGBRegressor(random_state=42, n_jobs=-1, verbosity=0),
    "LightGBM": LGBMRegressor(random_state=42, n_jobs=-1),
    "CatBoost": CatBoostRegressor(random_state=42, verbose=0)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print(f"{name} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")


XGBoost - MAE: 1145.02, RMSE: 1847.91
LightGBM - MAE: 1238.03, RMSE: 1897.96
CatBoost - MAE: 1163.85, RMSE: 1712.93


In [18]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

# Define parameter grid
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5],
    'iterations': [500, 1000]
}

cat_model = CatBoostRegressor(random_state=42, verbose=0)

grid_search = GridSearchCV(estimator=cat_model, param_grid=param_grid,
                           scoring='neg_root_mean_squared_error', cv=3, n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", -grid_search.best_score_)

# Evaluate on test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Test MAE: {mae:.2f}")
print(f"Test RMSE: {rmse:.2f}")


Best parameters: {'depth': 6, 'iterations': 1000, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
Best RMSE: 1994.654310024038
Test MAE: 1131.45
Test RMSE: 1699.45


In [19]:
from sklearn.model_selection import cross_val_score

cat_model_cv = CatBoostRegressor(**grid_search.best_params_, random_state=42, verbose=0)
cv_scores = cross_val_score(cat_model_cv, X, y, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

print("Cross-validation RMSE scores:", -cv_scores)
print("Mean CV RMSE:", -cv_scores.mean())


Cross-validation RMSE scores: [1791.22307351 1840.1481918  1693.39016685 1675.53277418 1784.16940656]
Mean CV RMSE: 1756.8927225789575


In [20]:
import numpy as np

y_log = np.log1p(y)  # log(Price + 1) to handle zeros if any

# Split as before
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

# Train model on log prices
cat_model_log = CatBoostRegressor(**grid_search.best_params_, random_state=42, verbose=0)
cat_model_log.fit(X_train, y_train_log)

# Predict and inverse transform
y_pred_log = cat_model_log.predict(X_test)
y_pred = np.expm1(y_pred_log)  # inverse of log1p

# Evaluate
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE after log-transform: {mae:.2f}")
print(f"RMSE after log-transform: {rmse:.2f}")


MAE after log-transform: 1093.91
RMSE after log-transform: 1697.54


In [24]:
import os
import joblib

# Create directory if not exists
os.makedirs('../model', exist_ok=True)

# Save CatBoost model
best_model.save_model('../model/catboost_flight_price_model.cbm')

# Save columns list
model_columns = list(df_encoded.drop('Price', axis=1).columns)
joblib.dump(model_columns, '../model/model_columns.pkl')


['../model/model_columns.pkl']

In [25]:
from catboost import CatBoostRegressor
import joblib
import pandas as pd
import numpy as np

# Load the saved model
model = CatBoostRegressor()
model.load_model('../model/catboost_flight_price_model.cbm')

# Load the saved columns
model_columns = joblib.load('../model/model_columns.pkl')


In [26]:
def preprocess_input(input_dict):
    """
    input_dict example:
    {
        'Airline': 'IndiGo',
        'Source': 'Delhi',
        'Destination': 'Cochin',
        'Total_Stops': '1 stop',
        'Date_of_Journey': '24/03/2019',
        'Dep_Time': '22:20',
        'Arrival_Time': '01:10 25/03/2019',
        'Duration': '2h 50m'
    }
    """
    # 1. Map Total_Stops
    stops_mapping = {
        'non-stop': 0,
        '1 stop': 1,
        '2 stops': 2,
        '3 stops': 3,
        '4 stops': 4
    }
    total_stops = stops_mapping.get(input_dict['Total_Stops'], 0)
    
    # 2. Date_of_Journey features
    date = pd.to_datetime(input_dict['Date_of_Journey'], format='%d/%m/%Y')
    journey_day = date.day
    journey_month = date.month
    
    # 3. Dep_Time features
    dep_time = pd.to_datetime(input_dict['Dep_Time'], format='%H:%M')
    dep_hour = dep_time.hour
    dep_minute = dep_time.minute
    
    # 4. Arrival_Time features
    try:
        arrival_time = pd.to_datetime(input_dict['Arrival_Time'], format='%H:%M %d/%m/%Y')
    except:
        arrival_time = pd.to_datetime(input_dict['Arrival_Time'], format='%H:%M')
    arrival_hour = arrival_time.hour
    arrival_minute = arrival_time.minute
    
    # 5. Duration in minutes
    def duration_to_minutes(duration):
        time_parts = duration.split()
        total_mins = 0
        for part in time_parts:
            if 'h' in part:
                total_mins += int(part.replace('h', '')) * 60
            if 'm' in part:
                total_mins += int(part.replace('m', ''))
        return total_mins
    
    duration_mins = duration_to_minutes(input_dict['Duration'])
    
    # 6. Create base dataframe with all features except one-hot
    base_dict = {
        'Total_Stops': total_stops,
        'journey_day': journey_day,
        'journey_month': journey_month,
        'dep_hour': dep_hour,
        'dep_minute': dep_minute,
        'arrival_hour': arrival_hour,
        'arrival_minute': arrival_minute,
        'duration_mins': duration_mins
    }
    
    df = pd.DataFrame([base_dict])
    
    # 7. One-hot encode categorical columns manually to match model_columns
    # For each categorical column and category, create a column and set 1 if it matches else 0
    
    categorical_columns = ['Airline', 'Source', 'Destination']
    
    for col in categorical_columns:
        unique_vals = [c for c in model_columns if c.startswith(col + '_')]
        for val in unique_vals:
            category = val.split('_', 1)[1]
            df[val] = 1 if input_dict[col] == category else 0
    
    # 8. Add any missing columns with 0 (if some columns were not set)
    missing_cols = set(model_columns) - set(df.columns)
    for col in missing_cols:
        df[col] = 0
    
    # 9. Reorder columns to match model input exactly
    df = df[model_columns]
    
    return df


In [27]:
# Example input
user_input = {
    'Airline': 'IndiGo',
    'Source': 'Delhi',
    'Destination': 'Cochin',
    'Total_Stops': '1 stop',
    'Date_of_Journey': '24/03/2019',
    'Dep_Time': '22:20',
    'Arrival_Time': '01:10 25/03/2019',
    'Duration': '2h 50m'
}

# Preprocess
input_df = preprocess_input(user_input)

# Predict price
predicted_price = model.predict(input_df)[0]

print(f"Predicted Flight Price: ₹{predicted_price:.2f}")


Predicted Flight Price: ₹6122.47
